In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
# two 3*3 convolution residual block
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.residual = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
 
    def forward(self, x):
        out = self.residual(x)
        out += self.shortcut(x)
        out = func.relu(out)
        return out

# Residual network
class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            # small initial convolution
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        #fc with dropout
        self.fc = nn.Linear(512, 512)
        self.drop = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(512, num_classes)
 
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
 
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = func.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.drop(out)
        out = self.fc2(out)
        return out


def ResNet18():
    return ResNet(ResidualBlock)

In [ ]:
# Data preprocessing
BATCH_SIZE = 128

# augmentation and normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), # fill with 0 around and crop
    transforms.RandomHorizontalFlip(),  # half chance to flip
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
 
transform_validate = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=True, download=True, transform=transform_train)
validset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=True, download=True, transform=transform_validate)

# split train and validation set
num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train))

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, sampler=train_sampler,num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE, sampler=valid_sampler,num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ', device)
if device = "cpu":
    print("Please delete all .cuda() and .cpu() while using a cpu.")

In [ ]:
# hyperparameters
EPOCH = 250
LR = 0.1
 
net = ResNet18().cuda()

# optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
# apply learning rate decay
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[135,185,240],gamma=0.1)

In [ ]:
# Training
print("Start Training, Resnet-18!")
for epoch in range(EPOCH):
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    data_num = 0
    for i, data in enumerate(trainloader, 0):
        # prepare data
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
 
        # forward and backword
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        data_num = i
    scheduler.step()
    print('[epoch:%d] Loss: %.03f | Acc: %.3f%% ' 
            % (epoch + 1, sum_loss / (data_num + 1), 100. * correct / total))
    
    # test accuracy
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for data in testloader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
        print('Test accuracy: %.3f%%' %(100 * correct_test / total_test))
        
print("Training Finished, TotalEPOCH=%d" % EPOCH)

In [ ]:
torch.save(net.state_dict(), './ResNet18.path')